In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"raghavendarkotha","key":"2d8e3375795968fd0ef2f321aafa715a"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d banudeep/nthuddd2

Dataset URL: https://www.kaggle.com/datasets/banudeep/nthuddd2
License(s): unknown
100% 2.80G/2.81G [01:23<00:00, 15.8MB/s]
100% 2.81G/2.81G [01:23<00:00, 36.1MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile("nthuddd2.zip", 'r') as zip_ref:
    zip_ref.extractall("nthu-dataset")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os
from tqdm import tqdm

In [ ]:
data_dir="/content/nthu-dataset/train_data"

In [ ]:
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)


In [ ]:
from collections import Counter
class_names = full_dataset.classes
class_to_idx = full_dataset.class_to_idx
class_counts = Counter(full_dataset.targets)
print("Number of images per class:")
for class_idx, count in class_counts.items():
    class_name = class_names[class_idx]
    print(f"  {class_name}: {count}")

Number of images per class:
  drowsy: 36030
  notdrowsy: 30491


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c, kernel_size=3, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(in_c, out_c, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_c)
        self.act = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, in_c):
        super().__init__()
        self.layer = nn.Sequential(
            ConvBlock(in_c, in_c // 2, 1),
            ConvBlock(in_c // 2, in_c, 3)
        )
    def forward(self, x):
        return x + self.layer(x)

class Darknet53(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            ConvBlock(3, 32),
            ConvBlock(32, 64, 3, 2),
            ResidualBlock(64)
        )
        self.layer2 = nn.Sequential(
            ConvBlock(64, 128, 3, 2),
            *[ResidualBlock(128) for _ in range(2)]
        )
        self.layer3 = nn.Sequential(
            ConvBlock(128, 256, 3, 2),
            *[ResidualBlock(256) for _ in range(8)]
        )
        self.layer4 = nn.Sequential(
            ConvBlock(256, 512, 3, 2),
            *[ResidualBlock(512) for _ in range(8)]
        )
        self.layer5 = nn.Sequential(
            ConvBlock(512, 1024, 3, 2),
            *[ResidualBlock(1024) for _ in range(4)]
        )

    def forward(self, x):
        for layer in [self.layer1, self.layer2, self.layer3, self.layer4, self.layer5]:
            x = layer(x)
        return x

class AttentionModule(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        attn = torch.softmax(x.view(x.size(0), -1), dim=1).view_as(x)
        return x + self.gamma * attn * x

class AttentionYOLOv3(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = Darknet53()
        self.attn = AttentionModule(1024)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        feat = self.backbone(x)
        feat = self.attn(feat)
        out = self.gap(feat).flatten(1)
        logits = self.fc(out)
        return logits


In [ ]:
files.upload()

Saving attention_yolov3_drowsy_5epochs.pth to attention_yolov3_drowsy_5epochs.pth


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttentionYOLOv3(num_classes=2).to(device)
model.load_state_dict(torch.load("/content/attention_yolov3_drowsy_5epochs.pth", map_location=device))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    avg_loss = total_loss / len(train_loader)
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_loss += criterion(outputs, labels).item()
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss /= len(val_loader)
    scheduler.step()

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

Epoch 1/2 [Train]: 100%|██████████| 6652/6652 [40:45<00:00,  2.72it/s]


Epoch [1/2] Train Loss: 0.3519 | Train Acc: 84.08% | Val Loss: 0.3704 | Val Acc: 83.38%


Epoch 2/2 [Train]: 100%|██████████| 6652/6652 [40:47<00:00,  2.72it/s]


Epoch [2/2] Train Loss: 0.2912 | Train Acc: 87.49% | Val Loss: 0.2543 | Val Acc: 89.03%


In [ ]:
os.makedirs("saved_models", exist_ok=True)
torch.save(model.state_dict(),"/attention_yolov3_drowsy_8epochs.pth")
print("✅ Model saved as saved_models/attention_yolov3_drowsy_8epochs.pth")

✅ Model saved as saved_models/attention_yolov3_drowsy_8epochs.pth


In [ ]:
torch.save(model.state_dict(),"/content/attention_yolov3_drowsy_8epochs.pth")